# 1.LDA主题模型理论

LDA latent Dirichlet Allocation

是一种无监督贝叶斯模型
是一种主题模型，它可以将文档集合中每篇文档的主题按照概率分布的形式给出。
同时它是一种无监督学习方法，在训练时不需要手工标注训练集，需要的仅仅是文档集以及指定主题的数量k即可。
**此外LDA的另一个优点则是，对于每一个主题均可找出一些词语来描述它（主题自描述）**。

是一种典型的词袋模型，即它认为一个文档是由一组词构成的一个集合，词与词之间没有顺序以及先后顺序的关系。一篇文档可以包含多个主题，文档中每个词都由其中的一个主题生成。

## 1.1.直观版

### 举例

假设某企业招聘工程师
收到一堆简历
希望直接通过简历筛选牛人

简历通常包含这些个人特征：百度实习、top2毕业。。。

特征--简历--优秀／不优秀

问题：你并不知道优秀的具有哪些特征，不优秀的具备哪些特征

1. 开始猜
2. 证实猜想
3. 反复1-2
4. 特征分类

### 理论

贝叶斯模型：

$P(Y,X) = P(Y)P(X|Y) = P(X)P(Y|X)$

$P(Y|X) = \frac{P(Y,X)}{P(X)}  = \frac{P(Y)P(X|Y)}{P(X)} = \frac{P(X)P(Y|X)}{P(X)}$

例如：$P(堵车|车祸) = \frac{P(堵车)P(车祸|堵车)}{P(车祸)}$

#### 贝叶斯学派：先验，后验，似然

* 先验概率：根据经验得到的结果的概率（已知结果） P(Y)
* 后验概率：在知道原因的情况下，求结果发生的概率（执因求果）P(Y|X)
* 似然概率：知道结果的情况下，求最可能导致结果发生的原因（知果求因）P(X|Y)


> 贝叶斯模型的伟大之处在于在条件概率计算中实现先验后验的转换。



## 1.2.标准版

> $P(Y|X) = \frac{P(Y,X)}{P(X)}  = \frac{P(Y)P(X|Y)}{P(X)}$

> $P(Y|Z) = \frac{P(Y,Z)}{P(Z)}  = \frac{P(Y)P(Z|Y)}{P(Z)}$

> $P(Z|X) = \frac{P(Z,X)}{P(X)}  = \frac{P(Z)P(X|Z)}{P(X)}$


> $P(Y|Z) * P(Z|X) = \frac{P(Y)P(Z|Y)}{P(Z)} * \frac{P(Z)P(X|Z)}{P(X)} =...=P(Y|X) $


一份简历中的每个特征都是因为候选人有一定概率是好／坏的程序员，并从好／坏这个分类中以一定概率选择某些特征组成。

一篇文章的每个词都是以一定概率选择了某个主题，并从这个主题中以一定的概率选择某个词语而组成。

> $P(单词|文档)=P(单词|主题)*P(主题|文档)$


### LDA生成过程

对于语料库中每篇文档，LDA定义如下生成过程（generative process）:
1. 对于每篇文档，从主题分布中抽取一个主题；
2. 从上述被抽到的主题所对应的单词分布中抽取一个单词；
3. 通过单词在文档中的分布（可观测）拟合验证；
4. 重复上述过程直至便利文档中的每一个单词。

具体：（w代表单词；d代表文档；t代表主题；大写代表总集合，小写代表个体）

* D中每个文档看作一个单词序列<w1,w2,w3,...,wn>,wi表斯第i个单词；
 
* D中涉及所有不同单词组成一个词汇表大集合V（vocabulary），LDA以文档集合D作为输入，希望训练出两个结果向量（设形成k个主题topic，V中共m个单词）；

 1. 对D中每个文档d，对应到k个不同topic的概率 Pd<pt1,pt2,pt3,...,ptk>，其中pti表示d对应T中第i个topic的概率。计算方法：pti=nti/n,nti表示d中对应第i个topic的词的数目，n是d中所有词的总数；
 2. 对T中每个主题t，生成不同单词的概率Pt<pw1,pw2,pw3,...,pwm>，其中pwi表示t生成V中第i个单词的概率。计算方法：pwi=Nwi/N,Nwi表示对应到t中的V中第i个单词的数目，N表示所有对应到t中单词的总数。
 
* 通过文章中出现单词的客观概率验证修正Pd 和 Pt
> P(w|d)=P(w|t) * P(t|d)

直观的看这个公式，就是以topic作为中间层，可以通过当前的Pd和Pt给出文档d中出现单词w的概率。其中p(t|d)通过Pd计算，p(w|t)通过Pt计算；
实际上利用Pd和Pt，我们可以为一个文档中的一个单词计算它对应任意一个topic的p(w|d)，然后根据这些结果更新这个词应该对应的topic。然后，如果这个更新改变了单词所对应的topic，就会反过来影响Pt 和 Pd。

### LDA学习过程

算法开始时先给Pt和Pd随机赋值；然后：
1. 针对一个特定文档d中的第i个单词wi，如果令该单词对应的topic为tj，可以把上述公式改写为：Pj(wi|d)=P(wi|tj) * P(tj|d)
2. 现在我们可以枚举T中的topic，得到所有的pj(wi|d)。然后可以根据这些概率值结果为d中第i个单词wi选择一个topic。最简单的想法是取令pj(wi|d)最大的tj(这里只有j是变量)
3. 然后，如果d中单词遍历过程中的第i个单词wi在这里选择了一个与原先不同的topic，就会对Pt和Pd有影响。它们的影响优惠反过来影响上面p(w|d)的计算。对D中所有d中的所有w进行一次p(w|d)计算并重新选择topic看作一次迭代。这样进行n次循环迭代就会收敛LDA想要的结果了。


## 1.3.公式版

### 共轭分布与共轭先验

后验概率（posterior probability） =》 似然函数（likelihood function）* 先验概率（prior probability）

### gamma函数

### 二项分布（Binomial distribution）

### 多项分布

### beta分布，二项分布的共轭先验分布

### Dirichlet分布，beta分布在高纬度上的推广

### 几个模型

* Unigram model
* mix unigram
* PLSA模型
* LDA 是贝叶斯版 PLSA

PLSA是频率派思想，LDA是贝叶斯派思想






# 2.LDA实战-希拉里邮件门

## 2.1.读取数据

In [ ]:
import numpy as np
import pandas as pd
import re

df = pd.read_csv("../input/HillaryEmails.csv")
# 去除Nan值
df = df[['Id','ExtractedBodyText']].dropna()

def clean_email_text(text):
    text = text.replace('\n','')
    # 把“-”两个单词分开 july-edu => july edu
    text = re.sub(r"-"," ",text)
    # 去除日期
    text = re.sub(r"\d+/\d+/\d+","",text)
    # 邮件地址
    text = re.sub(r"[\w]+@[\.\w]+", "", text)
    # 去网址
    text = re.sub(r"https?://[\.\w]+","", text)
    pure_text = ""
    for letter in text:
        # 只留下字母和空格
        if letter.isalpha() or letter==" ":
            pure_text += letter
            
    text = " ".join(word for word in pure_text.split() if len(word) > 1)
    return text

docs = df['ExtractBodyText']
docs = docs.apply(lambda s: clean_email_text(s))

docs.head(1).values

doclist = docs.values

## 2.2.LDA 模型构建

使用Gensim做模型构建
首先把刚刚整理出来的文本数据，转化为Gemsim认可的语料库形式：
[[a,bc,d],[e,fg,h]...]

In [ ]:
from gensim import corpora, models, similarities
import gensim

# 免去安装nltk，手写停止词列表
stoplist = ['vary','ourselves','am','']

# 分词
texts = [[word for word in doc.lower().split() if word not in stoplist] for doc in doclist]
texts[0]

In [ ]:
# 标记化，把每个单词用一个数字index指代，把原文本比变成一个个数组
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# 建立训练模型
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20)

# 查看10号主题
lda.print_topic(10, topn=5)

# 打印所有主题
lda.print_topics(num_topics=20, num_words=5)

# 使用模型对文本（标记数组化后）分类
bow = [1,2,3]
lda.get_document_topics(bow)

# 使用模型对单词分类
word_id = 2
lda.get_term_topics(word_id)